In [1]:
import pandas as pd
import numpy as np

import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import jax
import math

from fava.inference.fit import GaussianSKIMFA
from fava.basis.maps import LinearBasis, RepeatedFiniteBasis, TreeBasis
from fava.misc.scheduler import constantScheduler
from fava.misc.logger import GausLogger
from fava.decomposers.tensor_product import TensorProductKernelANOVA, LinearANOVA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import PolynomialFeatures


# Load and process data

In [2]:
data = pd.read_csv('../../breast_cancer.csv')
data = data.sample(frac=1, random_state=23) # shuffle all the data

In [3]:
X = data.drop(['diagnosis'], axis=1).values.copy()
Y = 2 * data['diagnosis'].apply(lambda x: x == 'M').astype(np.int32).values - 1 # -1 1 labels

In [4]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=.2, random_state=42)


# Random Forest Fit

In [5]:
f = RandomForestClassifier(n_estimators=5000, oob_score=True, random_state=32)
f.fit(X_train, Y_train)

print(f'Random Forest AUROC: {roc_auc_score(Y_valid, f.predict_proba(X_valid)[:, 1])}')
print(f'Random Forest Brier Score: {(( (Y_valid + 1) / 2 - f.predict_proba(X_valid)[:, 1]) ** 2).mean()}')

Random Forest AUROC: 0.997229916897507
Random Forest Brier Score: 0.020046255789473687


# SKIM-FA fit

In [6]:
X_train = jnp.array(X_train)
X_valid = jnp.array(X_valid)
Y_train = jnp.array(np.array(Y_train, dtype=np.float32))
Y_valid = jnp.array(np.array(Y_valid, dtype=np.float32))

key = random.PRNGKey(0)
p = X.shape[1]

kernel_params = dict()
Q = 1
kernel_params['U_tilde'] = jnp.ones(p)
kernel_params['eta'] = jnp.ones(Q+1)

hyperparams = dict()
hyperparams['sigma_sq'] = .5 #
hyperparams['c'] = 0.

opt_params = dict()
opt_params['cg'] = True
opt_params['cg_tol'] = .01
opt_params['M'] = 100
opt_params['gamma'] = .1
opt_params['T'] = 1000

featprocessor = LinearBasis(X_train)

scheduler = constantScheduler()
logger = GausLogger(100)

opt_params['scheduler'] = scheduler

skim = GaussianSKIMFA(X_train, Y_train, X_valid, Y_valid, featprocessor)

skim.fit(key, hyperparams, kernel_params, opt_params, 
            logger=GausLogger())

  0%|                                                                                                                | 0/1000 [00:00<?, ?it/s]

============================== Iteration 0/1000 ==============================
There are 30 covariates selected.


  0%|                                                                                                        | 1/1000 [00:03<56:37,  3.40s/it]

MSE (Validation)=0.20892752707004547.
R2 (Validation)=0.764956533908844.
eta=[0.9999941 1.0000097]
kappa=[0.4999523  0.49996924 0.49986637 0.5000332  0.4999959  0.5000249
 0.50002855 0.5000076  0.50002    0.49999544 0.50002396 0.4999767
 0.5000925  0.4995858  0.5000071  0.5000049  0.50002617 0.50003105
 0.50000054 0.4999902  0.50015247 0.50018716 0.49999863 0.49981734
 0.4999586  0.49999735 0.500196   0.4999951  0.5000445  0.50002503]


 10%|██████████▍                                                                                           | 102/1000 [00:11<01:16, 11.71it/s]

...

 80%|█████████████████████████████████████████████████████████████████████████████████▌                    | 800/1000 [01:06<00:15, 12.79it/s]

============================== Iteration 800/1000 ==============================
There are 30 covariates selected.
MSE (Validation)=0.21456177532672882.
R2 (Validation)=0.7586179971694946.
eta=[0.99425656 1.3325282 ]
kappa=[0.46282974 0.46446425 0.37392107 0.4631202  0.48727924 0.5045488
 0.5117285  0.50343716 0.5158147  0.49923712 0.6166246  0.5050145
 0.572452   0.02312083 0.5110954  0.4756329  0.5153906  0.49538684
 0.500466   0.48871168 0.5730418  0.59569407 0.48134163 0.44260147
 0.4542522  0.49636623 0.58981514 0.49846232 0.5291336  0.51469916]


 90%|████████████████████████████████████████████████████████████████████████████████████████████          | 902/1000 [01:14<00:08, 11.48it/s]

============================== Iteration 900/1000 ==============================
There are 30 covariates selected.
MSE (Validation)=0.21482402086257935.
R2 (Validation)=0.7583229541778564.
eta=[0.986191  1.3689477]
kappa=[0.4548454  0.45924145 0.3499472  0.45663518 0.4828022  0.49993935
 0.5068464  0.50084585 0.5177235  0.49947083 0.6290277  0.5094496
 0.580177   0.01792253 0.5110628  0.46418282 0.5223138  0.4935576
 0.5004358  0.4879491  0.5812498  0.60434693 0.47803554 0.43345958
 0.44445723 0.4956367  0.59391063 0.49831104 0.5321527  0.51540196]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:22<00:00, 12.15it/s]


In [7]:
print(f'SKIM-FA AUROC: {roc_auc_score(Y_valid, skim.predict(X_valid))}')
print(f'SKIM-FA Brier Score: {((Y_valid - skim.predict(X_valid)) ** 2).mean()}')


SKIM-FA AUROC: 0.997229916897507
SKIM-FA Brier Score: 0.21482402086257935


# Since we treat the classification task as a regression task, SKIM-FA has poor probability estimates even though it has high discrimination (AUROC nearly 1). To correct for this bias, we use Platt scaling.

In [8]:
# We should really use 5-fold CV for platt scaling here...
from sklearn.linear_model import LogisticRegression

In [9]:
platt = LogisticRegression(penalty=None, fit_intercept=True)
skim_train_scores = np.array(skim.predict(X_train)).reshape((X_train.shape[0],1))
skim_val_scores = np.array(skim.predict(X_valid)).reshape((X_valid.shape[0],1))

In [10]:
clf = LogisticRegression(random_state=0).fit(skim_train_scores, np.array(Y_train))

In [11]:
print(f'SKIM-FA Platt Brier Score: {(( (Y_valid + 1) / 2 - clf.predict_proba(skim_val_scores)[:, 1]) ** 2).mean()}')


SKIM-FA Platt Brier Score: 0.02111034467816353


In [12]:
from sklearn.calibration import calibration_curve

In [13]:
calibration_curve((Y_valid + 1) / 2, clf.predict_proba(skim_val_scores)[:, 1], strategy='quantile')

(array([0.        , 0.        , 0.        , 0.65217391, 1.        ]),
 array([0.0017444 , 0.01274638, 0.04941643, 0.57116257, 0.99684133]))